In [1]:
# Clean and Transform Spotify data
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import time

SpotifyExcelFile = '../Spotify/SpotifyPlaylistData_20230109.xlsx'

def cleanPlaylistData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)

  df = df.drop(columns=['Unnamed: 0', 'duration_ms'])

  df = df.loc[:,['track_uri','artist_uri', 'album_uri', 'track_name', 'artist_name', 'album_name', 'release_date', 'explicit', 'popularity']]
  # df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')
  df = df.reset_index(drop=True)
  
  return df

def cleanArtistData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df = df.reset_index(drop=True)
  
  return df

def cleanAlbumData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0', 'artist_uri'])
  df = df.loc[:,['album_uri', 'album_name', 'artist_name', 'release_date', 'label', 'total_tracks', 'popularity']]
  df = df.reset_index(drop=True)
  
  return df

def cleanTrackFeaturesData(sheetName):

  df = pd.read_excel(SpotifyExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df.rename(columns={'uri': 'track_uri'}, inplace=True)
  df = df.reset_index(drop=True)
  
  return df


def mergeDataFrames(HHDF, CHHDF):
  mergedDF = pd.concat([HHDF, CHHDF], ignore_index=True)

  return mergedDF

HHPlaylistSpotDF = cleanPlaylistData('HHPlaylist')
CHHPlaylistSpotDF = cleanPlaylistData('CHHPlaylist')
HHPlaylistSpotDF.insert(7, 'genre', 'Hip-Hop')
CHHPlaylistSpotDF.insert(7, 'genre', 'Christian Hip-Hop')

PlaylistSpotDF = mergeDataFrames(HHPlaylistSpotDF, CHHPlaylistSpotDF)

HHArtistsSpotDF = cleanArtistData('HHArtists')
CHHArtistsSpotDF = cleanArtistData('CHHArtists')

ArtistsSpotDF = mergeDataFrames(HHArtistsSpotDF, CHHArtistsSpotDF)

HHAlbumsSpotDF = cleanAlbumData('HHAlbums')
CHHAlbumsSpotDF = cleanAlbumData('CHHAlbums')

AlbumsSpotDF = mergeDataFrames(HHAlbumsSpotDF, CHHAlbumsSpotDF)

HHTrackFeaturesSpotDF = cleanTrackFeaturesData('HHTrackFeatures')
CHHTrackFeaturesSpotDF = cleanTrackFeaturesData('CHHTrackFeatures')

TrackFeaturesSpotDF = mergeDataFrames(HHTrackFeaturesSpotDF, CHHTrackFeaturesSpotDF)


AlbumsSpotDF


,album_uri,album_name,artist_name,release_date,label,total_tracks,popularity
0,spotify:album:5MS3MvWHJ3lOZPLiMxzOU6,Her Loss,Drake,2022-11-04,OVO / Republic Records,16,93
1,spotify:album:7txGsnDSqVMoRl6RQ9XyZP,HEROES & VILLAINS,Metro Boomin,2022-12-02,Republic Records,15,93
2,spotify:album:4lXLULUDQvqV1yHtLW5fyL,Lil Durk Presents: Loyal Bros 2,Only The Family,2022-12-16,Only The Family / EMPIRE,23,72
3,spotify:album:1hcA90r3ikdVN4ylEz204T,Quickie,Moneybagg Yo,2022-12-08,CMG/N-Less/Interscope Records,1,65
4,spotify:album:0TDPqwRP3NuPYnxm45BqPg,My All,Polo G,2022-12-09,Columbia,1,69
...,...,...,...,...,...,...,...
88,spotify:album:03gUkPWtNaECPwgUzyDWnY,Can I Be Honest?,Dru Bex,2022-07-15,6 Luv Music,4,10
89,spotify:album:3FKY4dtLXzcRTdcgUenvTg,AWAKEN ME II,L. Dejuan,2022-10-28,Love Driven Ent.,16,10
90,spotify:album:3lS1mZLFr8SQkfyZ8zY7Fi,See,Solachi Voz,2022-11-01,TBTLA MUSIC,1,1
91,spotify:album:2MRsC1Zuo9ZOBUNq0CPReu,I C O N I C,Tre'Gadd,2022-11-04,Tre'Gadd,1,14


In [2]:
# Clean and Transform LastFM data
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import time

LastFMExcelFile = '../LastFM/LastFMData_20230109.xlsx'

def cleanLastFMTrackData(sheetName):

  df = pd.read_excel(LastFMExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  df['artist_mbid'] = df['artist_mbid'].replace(np.nan, None)
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  # df = df.sort_values(by='artist_name', ascending=False)
  # df = df.sort_values(by='playcount', ascending=False)
  df = df.reset_index(drop=True)
  
  return df

def cleanLastFMArtistData(sheetName):

  df = pd.read_excel(LastFMExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  df = df.drop_duplicates(['artist_name'])
  df['artist_mbid'] = df['artist_mbid'].replace(np.nan, None)
  # df = df.sort_values(by='artist_name', ascending=False)
  # df = df.sort_values(by='playcount', ascending=False)
  df = df.reset_index(drop=True)
  
  return df

def cleanLastFMAlbumData(sheetName):

  df = pd.read_excel(LastFMExcelFile,sheet_name=sheetName)
  df = df.drop(columns=['Unnamed: 0'])
  # df = df.loc[:,['artist_mbid', 'track_name', 'artist_name', 'listeners', 'playcount']]
  df = df.drop_duplicates(subset=['album_mbid', 'album_name'])
  df['album_mbid'] = df['album_mbid'].replace(np.nan, None)
  # df = df.sort_values(by='listeners', ascending=False)
  # df = df.sort_values(by='playcount', ascending=False)
  df = df.reset_index(drop=True)
  
  return df

def mergeDataFrames(HHDF, CHHDF):
  mergedDF = pd.concat([HHDF, CHHDF], ignore_index=True)

  return mergedDF

HHAlbumLastDF = cleanLastFMAlbumData('HHAlbumData')
CHHAlbumLastDF = cleanLastFMAlbumData('CHHAlbumData')
AlbumsLastDF = mergeDataFrames(HHAlbumLastDF, CHHAlbumLastDF)

HHTrackLastDF = cleanLastFMTrackData('HHTrackData')
CHHTrackLastDF = cleanLastFMTrackData('CHHTrackData')
TrackLastDF = mergeDataFrames(HHTrackLastDF, CHHTrackLastDF)

HHArtistLastDF = cleanLastFMArtistData('HHArtistData')
CHHArtistLastDF = cleanLastFMArtistData('CHHArtistData')
ArtistLastDF = mergeDataFrames(HHArtistLastDF, CHHArtistLastDF)

# HHArtistLastDF.head()
# HHAlbumLastDF.head()
# TrackLastDF.head()

AlbumsLastDF

,album_mbid,album_name,listeners,playcount
0,None,Her Loss,510432,20471616
1,66de5e14-0642-4803-998c-3ecdf504134d,Heroes & Villains,398032,18060159
2,None,Lil Durk Presents: Loyal Bros 2,551,2859
3,None,Quickie,14590,43598
4,None,My All,33144,183005
...,...,...,...,...
89,None,Can I Be Honest?,24,66
90,None,AWAKEN ME II,19,156
91,None,See,1,1
92,None,I C O N I C,31,102


In [3]:
# Join Spotify and LastFM data

joinedPlaylisttDF = PlaylistSpotDF.join(TrackLastDF, lsuffix='_Spotify', rsuffix='_LastFM')
joinedPlaylisttDF = joinedPlaylisttDF.drop(columns=['artist_mbid', 'artist_name_LastFM', 'track_name_LastFM', 'album_name_LastFM'])
joinedPlaylisttDF.rename(columns={'track_name_Spotify': 'track_name', 'artist_name_Spotify': 'artist_name', 'album_name_Spotify': 'album_name', 'listeners': 'listeners_LastFM', 'playcount': 'playcount_LastFM'}, inplace=True)

joinedArtistsDF = ArtistsSpotDF.join(ArtistLastDF, lsuffix='_Spotify', rsuffix='_LastFM')
joinedArtistsDF = joinedArtistsDF.drop(columns=['artist_mbid', 'artist_name_LastFM'])
joinedArtistsDF.rename(columns={'artist_name_Spotify': 'artist_name', 'listeners': 'listeners_LastFM', 'playcount': 'playcount_LastFM'}, inplace=True)

joinedAlbumsDF = AlbumsSpotDF.join(AlbumsLastDF, lsuffix='_Spotify', rsuffix='_LastFM')
joinedAlbumsDF = joinedAlbumsDF.drop(columns=['album_mbid', 'album_name_LastFM'])
joinedAlbumsDF.rename(columns={'album_name_Spotify': 'album_name', 'listeners': 'listeners_LastFM', 'playcount': 'playcount_LastFM'}, inplace=True)

joinedPlaylisttDF

,track_uri,artist_uri,album_uri,track_name,artist_name,album_name,release_date,genre,explicit,popularity,listeners_LastFM,playcount_LastFM
0,spotify:track:1bDbXMyjaUIooNwFE9wn0N,spotify:artist:3TVXtAsR1Inumwj472S9r4,spotify:album:5MS3MvWHJ3lOZPLiMxzOU6,Rich Flex,Drake,Her Loss,2022-11-04,Hip-Hop,True,96,420763,3341051
1,spotify:track:0vjeOZ3Ft5jvAi9SBFJm1j,spotify:artist:0iEtIxbK0KxaSlF7G42ZOp,spotify:album:7txGsnDSqVMoRl6RQ9XyZP,Superhero (Heroes & Villains) [with Future & C...,Metro Boomin,HEROES & VILLAINS,2022-12-02,Hip-Hop,True,90,222892,2031645
2,spotify:track:2i2qDe3dnTl6maUE31FO7c,spotify:artist:3hcs9uc56yIGFCSy9leWe7,spotify:album:4lXLULUDQvqV1yHtLW5fyL,Mad Max,Lil Durk,Lil Durk Presents: Loyal Bros 2,2022-12-16,Hip-Hop,True,79,32169,105365
3,spotify:track:3084aAzars6DR26nCWO9Ep,spotify:artist:3tJoFztHeIJkJWMrx0td2f,spotify:album:1hcA90r3ikdVN4ylEz204T,Quickie,Moneybagg Yo,Quickie,2022-12-08,Hip-Hop,True,77,14731,44058
4,spotify:track:2Hh3ETdQKrmSI3QS0hme7g,spotify:artist:0iEtIxbK0KxaSlF7G42ZOp,spotify:album:7txGsnDSqVMoRl6RQ9XyZP,Too Many Nights (feat. Don Toliver & with Future),Metro Boomin,HEROES & VILLAINS,2022-12-02,Hip-Hop,True,87,195646,1640164
...,...,...,...,...,...,...,...,...,...,...,...,...
95,spotify:track:3o2UJytsbDMXcv7G8SH2kW,spotify:artist:11hR4oPRwSDzaUMTPzYuoe,spotify:album:03gUkPWtNaECPwgUzyDWnY,Reality Checks,Dru Bex,Can I Be Honest?,2022-07-15,Christian Hip-Hop,False,12,76,170
96,spotify:track:6TTtmmoLlZqMJ4PilWYttZ,spotify:artist:7uL0ibr4zltGHlcMe8EWBg,spotify:album:3FKY4dtLXzcRTdcgUenvTg,PRIORITIES,L. Dejuan,AWAKEN ME II,2022-10-28,Christian Hip-Hop,False,3,15,29
97,spotify:track:48jSEBP7d6mSdSjEPWfWA5,spotify:artist:0rAGfuDA6xM9nZ0Rz751Gy,spotify:album:3lS1mZLFr8SQkfyZ8zY7Fi,See,Solachi Voz,See,2022-11-01,Christian Hip-Hop,False,4,1,1
98,spotify:track:07Uergj3vqMSMu0XHWsLWF,spotify:artist:6NCcfV9K0gZqFBfFLQqWRH,spotify:album:2MRsC1Zuo9ZOBUNq0CPReu,I C O N I C,Tre'Gadd,I C O N I C,2022-11-04,Christian Hip-Hop,False,26,32,112


In [4]:
# Push data to excel

import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import datetime as datetime

xlxWriter = pd.ExcelWriter('TransformedMusicData.xlsx', engine='xlsxwriter')

joinedPlaylisttDF.to_excel(xlxWriter, sheet_name='Playlist')
joinedArtistsDF.to_excel(xlxWriter, sheet_name='Artists')
joinedAlbumsDF.to_excel(xlxWriter, sheet_name='Albums')
TrackFeaturesSpotDF.to_excel(xlxWriter, sheet_name='TrackFeatures')

xlxWriter.close()

In [5]:
# Push data to SQL Server
import pyodbc
import sqlalchemy
from sqlalchemy.engine import URL
from sqlalchemy.types import SmallInteger, Text, String, DateTime, Boolean, Integer, Float
import os
from dotenv import load_dotenv
import pandas as pd
import json

load_dotenv()

LOCAL_SERVER_NAME = os.getenv('LOCAL_SERVER_NAME')
dbName = 'TrackStarzMusicAnalysis'

connection_string = ("Driver={SQL Server Native Client 11.0};"
            f"Server={LOCAL_SERVER_NAME};"
            f"Database={dbName};"
            "Trusted_Connection=yes")
      
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = sqlalchemy.create_engine(connection_url)

conn = engine.connect()

def createSQLTable(dataFrame, tableName, dataTypes):
  dataFrame.to_sql(tableName, 
    con=conn, 
    if_exists='replace', 
    index=False, 
    dtype=dataTypes)

createSQLTable(joinedPlaylisttDF, 'SpotifyPlaylist', {
    "track_uri": String(50),
    "artist_uri": String(50),
    "album_uri": String(50),
    "track_name": String(100),
    "artist_name": String(100),
    "album_name": String(100),
    "release_date": DateTime(),
    "explicit": SmallInteger(),
    "popularity": SmallInteger(),
    "listeners_LastFM": Integer(),
    "playcount_LastFM": Integer()
})

createSQLTable(joinedArtistsDF, 'SpotifyArtists', {
    "artist_uri": String(50),
    "artist_name": String(100),
    "popularity": SmallInteger(),
    "followers": Integer(),
    "listeners_LastFM": Integer(),
    "playcount_LastFM": Integer()
})

conn.execute('ALTER TABLE SpotifyArtists ALTER COLUMN artist_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyArtists ADD PRIMARY KEY (artist_uri);')

createSQLTable(joinedAlbumsDF, 'SpotifyAlbums', {
    "album_uri": String(50),
    "album_name": String(100),
    "artist_name": String(100),
    "release_date": DateTime(),
    "label": String(100),
    "total_tracks": SmallInteger(),
    "popularity": SmallInteger(),
    "listeners_LastFM": Integer(),
    "playcount_LastFM": Integer()
})

conn.execute('ALTER TABLE SpotifyAlbums ALTER COLUMN album_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyAlbums ADD PRIMARY KEY (album_uri);')

createSQLTable(TrackFeaturesSpotDF, 'SpotifyTrackFeatures', {
    "track_uri": String(50),
    "track_name": String(100),
    "duration_ms": Integer(),
    "time_signature": SmallInteger(),
    "tempo": Float(),
    "valence": Float(),
    "liveness": Float(),
    "instrumentalness": Float(),
    "acousticness": Float(),
    "speechiness": Float(),
    "mode": SmallInteger(),
    "loudness": Float(),
    "key": SmallInteger(),
    "energy": Float(),
    "danceability": Float()
})

conn.execute('ALTER TABLE SpotifyTrackFeatures ALTER COLUMN track_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyTrackFeatures ADD PRIMARY KEY (track_uri);')


conn.execute('ALTER TABLE SpotifyPlaylist ALTER COLUMN track_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyPlaylist ADD PRIMARY KEY (track_uri);')
conn.execute('ALTER TABLE SpotifyPlaylist ALTER COLUMN artist_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyPlaylist ADD FOREIGN KEY (artist_uri) REFERENCES SpotifyArtists(artist_uri);')
conn.execute('ALTER TABLE SpotifyPlaylist ALTER COLUMN album_uri VARCHAR(50) NOT NULL;')
conn.execute('ALTER TABLE SpotifyPlaylist ADD FOREIGN KEY (album_uri) REFERENCES SpotifyAlbums(album_uri);')

conn.close()

Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 8
